In [ ]:
pip install ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets

# Function to model Susceptible, Infected, and Recovered populations using sigmoid and Gaussian functions
def flu_model(infection_spread_rate, recovery_rate, peak_infection_day, total_days, population_size):
    # Simulating over a user-defined number of days with 100 points
    days = np.linspace(0, total_days, 100)

    # Recovered Population: Sigmoid function (S-shaped curve)
    # The sigmoid function models the gradual recovery over time, scaled by the total population size.
    recovered_population = (1 / (1 + np.exp(-recovery_rate * (days - total_days / 2)))) * population_size

    # Infected Population: Gaussian (bell-shaped curve)
    # This models the infected population, peaking at 'peak_infection_day', scaled by the total population.
    infected_population = (np.exp(-infection_spread_rate * (days - peak_infection_day)**2)) * population_size

    # Susceptible Population: Remaining population that has not been infected or recovered
    susceptible_population = population_size - (infected_population + recovered_population)

    # Ensure no negative values for susceptible population due to floating-point errors
    susceptible_population[susceptible_population < 0] = 0

    # Plotting the results
    plt.figure(figsize=(10, 6))

    # Plot the susceptible population (Healthy, not yet infected)
    plt.plot(days, susceptible_population, label='Susceptible Population', color='orange')

    # Plot the infected population (Currently infected individuals)
    plt.plot(days, infected_population, label='Infected Population', color='red')

    # Plot the recovered population (People who have recovered)
    plt.plot(days, recovered_population, label='Recovered Population', color='green')

    # Adding labels, title, and legend for better understanding
    plt.title(f'Flu Model: Susceptible, Infected, and Recovered Populations Over Time (Population: {population_size})')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

# Interact function with sliders for parameters, allowing interactive control of the simulation
interact(flu_model,
         infection_spread_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Infection Spread Rate'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         peak_infection_day=FloatSlider(min=1, max=20, step=1, value=5, description='Infection Peak Day'),
         total_days=IntSlider(min=10, max=60, step=1, value=30, description='Total Days'),
         population_size=IntSlider(min=100, max=1000000, step=1000, value=10000, description='Population Size'));


interactive(children=(FloatSlider(value=0.2, description='Infection Spread Rate', max=1.0, min=0.01, step=0.01…

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# Function to simulate and visualize infection spread with recovery and contagiousness curves
def simulate_infection_graph(population_size, initial_infected, R0, recovery_rate, contagious_factor, days):
    G = nx.erdos_renyi_graph(population_size, 0.05)  # Creating a random graph

    # Initialize node states (susceptible by default)
    for node in G.nodes:
        G.nodes[node]['state'] = 'susceptible'

    # Infect a few initial nodes
    infected_nodes = np.random.choice(list(G.nodes), initial_infected, replace=False)
    for node in infected_nodes:
        G.nodes[node]['state'] = 'infected'

    # Initialize layout for consistent node positions
    pos = nx.spring_layout(G)

    # Simulate infection spread over multiple days
    for day in range(days):
        # Apply contagiousness based on the Gaussian model
        contagiousness = np.exp(-contagious_factor * (day - 5)**2)  # Gaussian for contagiousness

        new_infected = []
        for node in G.nodes:
            if G.nodes[node]['state'] == 'infected':
                for neighbor in G.neighbors(node):
                    if G.nodes[neighbor]['state'] == 'susceptible':
                        # Contagiousness controls infection probability
                        if np.random.rand() < R0 * contagiousness / 10:
                            new_infected.append(neighbor)

        # Infect new nodes
        for node in new_infected:
            G.nodes[node]['state'] = 'infected'

        # Recover some infected nodes based on the recovery rate
        recover_nodes = np.random.choice(infected_nodes, int(len(infected_nodes) * recovery_rate), replace=False)
        for node in recover_nodes:
            G.nodes[node]['state'] = 'recovered'

        infected_nodes = [node for node in G.nodes if G.nodes[node]['state'] == 'infected']

    # Final visualization after all steps
    plt.figure(figsize=(8, 8))
    node_colors = ['red' if G.nodes[node]['state'] == 'infected' else 'green' if G.nodes[node]['state'] == 'recovered' else 'blue' for node in G.nodes]
    nx.draw(G, pos, node_color=node_colors, with_labels=False, node_size=100)
    plt.title(f'Final Infection Spread after {days} Days')

    # Adding a legend manually
    plt.scatter([], [], c='red', label='Infected')
    plt.scatter([], [], c='blue', label='Susceptible')
    plt.scatter([], [], c='green', label='Recovered')
    plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc="upper right", fontsize='large')

    plt.show()

    # Short explanation
    print(f"Simulation complete after {days} days.")
    print("Explanation:")
    print("1. Red nodes: represent infected individuals who can spread the disease to their neighbors.")
    print("2. Blue nodes: represent susceptible individuals who have not yet been infected.")
    print("3. Green nodes: represent recovered individuals who can no longer spread or catch the disease.")
    print("\nThe graph simulates how an infection spreads through a population over time, with a few initial infected individuals.")
    print("As time progresses, susceptible individuals (blue) may become infected (red), and infected individuals recover (green) based on the recovery rate.")
    print("Parameters such as R0, recovery rate, and contagiousness control the dynamics of the infection.")

# Interactive sliders for the simulation parameters
interact(simulate_infection_graph,
         population_size=IntSlider(min=50, max=500, step=10, value=100, description='Population Size'),
         initial_infected=IntSlider(min=1, max=50, step=1, value=5, description='Initial Infected'),
         R0=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='R0 (Infection Rate)'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         contagious_factor=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Contagious Factor'),
         days=IntSlider(min=1, max=20, step=1, value=10, description='Days'));


interactive(children=(IntSlider(value=100, description='Population Size', max=500, min=50, step=10), IntSlider…